In [1]:
!date
!python --version

Thu Jun  6 03:11:24 AM UTC 2024
Python 3.10.12


# トピックモデルによるクラスタリング
トピックモデルとは文書中の単語出現分布を元に傾向（≒トピックらしきもの）を観察しようとするアプローチで、クラスタリングの一種である。なお、一般的なクラスタリング（例えば[k平均法](https://ja.wikipedia.org/wiki/K平均法)）では一つのサンプルが一つのクラスタに属するという前提でグルーピングを行うのに対し、トピックモデルでは一つのサンプルが複数のクラスタを内包しているという前提でグルーピングを行う。次の例を眺めるとイメージをつかみやすいだろう。

- 例1: [トピックモデル入門：WikipediaをLDAモデル化してみた](https://recruit.gmo.jp/engineer/jisedai/blog/topic-model/)
- 例2: [Wikipedia: Topic model](https://en.wikipedia.org/wiki/Topic_model)

基本的には文書を BoW (CountVectrizor) やそれの重みを調整した TF-IDF 等の「文書単語行列」を作成し、ここから文書館類似度や単語間類似度を元に集約（≒次元削減）を試みる。文書単語行列の作成方法や次元削減方法、類似度の求め方などで様々なアルゴリズムが提案されている。ここでは (1) Bow + [LDA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html) によりトピックモデルを行い、[PyLDAvis](https://github.com/bmabey/pyLDAvis)による可視化を通してトピックを観察してみよう。

なお、トピックモデルの注意点として、トピックそのものは人手による解釈が求められる点が挙げられる。例えば先に上げた[トピックモデル入門：WikipediaをLDAモデル化してみた](https://recruit.gmo.jp/engineer/jisedai/blog/topic-model/)における図2（下図）では「政治」「スポーツ」「国際」といったトピックが並んでいるが、実際には「4-1. トピック観察」を行う必要がある。実際に観察してみよう。

In [2]:
# spacy, ginzaインストール
!pip install -U ginza ja_ginza

# PyLDAvis
!pip install pyldavis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 11.0 MB/s eta 0:00:00


## データの準備
これまで見てきたいつものやつ。

In [3]:
!curl -O https://ie.u-ryukyu.ac.jp/~tnal/2022/dm/static/r_assesment.pkl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 34834  100 34834    0     0  15859      0  0:00:02  0:00:02 --:--:-- 15862


In [4]:
import collections

import numpy as np
import pandas as pd
import spacy
from wordcloud import WordCloud

nlp = spacy.load("ja_ginza")

assesment_df = pd.read_pickle('r_assesment.pkl')
assesment_df.head()

,title,grade,required,q_id,comment
0,工業数学Ⅰ,1,True,Q21 (1),特になし
1,工業数学Ⅰ,1,True,Q21 (2),正直わかりずらい。むだに間があるし。
2,工業数学Ⅰ,1,True,Q21 (2),例題を取り入れて理解しやすくしてほしい。
3,工業数学Ⅰ,1,True,Q21 (2),特になし
4,工業数学Ⅰ,1,True,Q21 (2),スライドに書く文字をもう少しわかりやすくして欲しいです。


In [5]:
# 分かち書き
poses = ['PROPN', 'NOUN', 'VERB', 'ADJ', 'ADV'] #名詞、動詞、形容詞、形容動詞

assesment_df['wakati'] = ''
for index, comment in enumerate(assesment_df['comment']):
    doc = nlp(comment)
    wakati_words = []
    for token in doc:
        if token.pos_ in poses:
            wakati_words.append(token.lemma_)
    wakati_text = ' '.join(wakati_words)
    assesment_df.at[index, 'wakati'] = wakati_text

assesment_df

,title,grade,required,q_id,comment,wakati
0,工業数学Ⅰ,1,True,Q21 (1),特になし,特に なし
1,工業数学Ⅰ,1,True,Q21 (2),正直わかりずらい。むだに間があるし。,正直 わかる ずらい むだ 間 ある
2,工業数学Ⅰ,1,True,Q21 (2),例題を取り入れて理解しやすくしてほしい。,例題 取り入れる 理解 する
3,工業数学Ⅰ,1,True,Q21 (2),特になし,特に なし
4,工業数学Ⅰ,1,True,Q21 (2),スライドに書く文字をもう少しわかりやすくして欲しいです。,スライド 書く 文字 もう 少し わかる する
...,...,...,...,...,...,...
165,データマイニング,3,False,Q22,課題が難しいものが多く、時間を多くとってもらえたのは非常に良かったですがかなりきつかったです...,課題 難しい もの 多い 時間 多い とる もらえる 非常 良い かなり きつい ござる
166,ICT実践英語Ⅰ,3,False,Q22,オンラインなどで顔を合わせてやりたかったです。,オンライン 顔 合わせる やる
167,知能情報実験Ⅲ,3,True,Q21 (2),unityの操作方法の説明などを最初に行ってもらえたらもう少しスムーズにできたのではないかと思う。,unity 操作方法 説明 最初 行く もらえる もう 少し スムーズ できる 思う
168,知能情報実験Ⅲ,3,True,Q22,それぞれに任せるといった形で進められたものだったのでそれなりに進めやすかったですが、オンライ...,それぞれ 任せる いう 形 進める もの なり 進める オンライン 班 員 指導 全く する...


## 文書ベクトルの作成
ここでは CountVectorizer (Bag-of-Words) で作成してみよう。

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

stop_words = ['こと', '\r\n', 'ため', '思う', 'いる', 'ある', 'する', 'なる']
vectorizer = CountVectorizer(stop_words=stop_words)
bow_tf_vector = vectorizer.fit_transform(assesment_df['wakati'])
print('bow_tf_vector.shape = ', bow_tf_vector.shape)

bow_tf_vector.shape =  (170, 740)


## LDAによるトピックモデル解析
sklearnでは [LatentDirichletAllocation](https://scikit-learn.org/stable/modules/decomposition.html?highlight=lda#latent-dirichlet-allocation-lda) として用意されている。

In [7]:
from sklearn.decomposition import LatentDirichletAllocation

NUM_TOPICS = 20 #トピック数
max_iter = 100  #LDAによる学習回数
lda = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=max_iter, learning_method='online',verbose=True)
data_lda = lda.fit_transform(bow_tf_vector)

iteration: 1 of max_iter: 100
iteration: 2 of max_iter: 100
iteration: 3 of max_iter: 100
iteration: 4 of max_iter: 100
iteration: 5 of max_iter: 100
iteration: 6 of max_iter: 100
iteration: 7 of max_iter: 100
iteration: 8 of max_iter: 100
iteration: 9 of max_iter: 100
iteration: 10 of max_iter: 100
iteration: 11 of max_iter: 100
iteration: 12 of max_iter: 100
iteration: 13 of max_iter: 100
iteration: 14 of max_iter: 100
iteration: 15 of max_iter: 100
iteration: 16 of max_iter: 100
iteration: 17 of max_iter: 100
iteration: 18 of max_iter: 100
iteration: 19 of max_iter: 100
iteration: 20 of max_iter: 100
iteration: 21 of max_iter: 100
iteration: 22 of max_iter: 100
iteration: 23 of max_iter: 100
iteration: 24 of max_iter: 100
iteration: 25 of max_iter: 100
iteration: 26 of max_iter: 100
iteration: 27 of max_iter: 100
iteration: 28 of max_iter: 100
iteration: 29 of max_iter: 100
iteration: 30 of max_iter: 100
iteration: 31 of max_iter: 100
iteration: 32 of max_iter: 100
iteration: 33 of 

## トピックの観察。
- [pyLDAvis](https://github.com/bmabey/pyLDAvis)によりトピックを観察してみよう。
- 下図の左側がトピック分布を表している。丸の大きさがトピック内に含まれる文書数、丸と丸の距離はトピック間の距離。
- 下図の右側が単語の発生頻度を表している。
  - トピックを選択するとそのトピックにおける単語の発生頻度を観察できる。
  - 単語を選択すると、その単語がどのようにトピック分布上にバラけているかを観察できる。

In [ ]:
import pyLDAvis.lda_model

In [9]:
import pyLDAvis
from pyLDAvis import lda_model

pyLDAvis.enable_notebook()
dash = lda_model.prepare(lda, bow_tf_vector, vectorizer, mds='tsne')
dash

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
14    -128.204605  -73.728752       1        1  53.340700
16     -25.731833  273.743103       2        1   8.888549
8     -172.758301   49.549717       3        1   7.389835
10    -140.080338  191.974487       4        1   4.121394
15      27.686457  151.515656       5        1   3.507018
9      202.026978   45.057007       6        1   2.828444
17      78.453629   28.120079       7        1   2.666101
4     -253.791046  -67.026825       8        1   2.596971
3       55.223969 -263.009918       9        1   2.467206
6     -108.504471 -295.491272      10        1   1.696712
18      -8.585955  -56.882542      11        1   1.690827
19    -271.293854  139.392059      12        1   1.621914
7      195.453903 -205.140854      13        1   1.581493
1     -358.715942    9.534597      14        1   1.288830
5      -47.121204 -174.312210      15        1   1.181598
13     160.504410  183.042816      16        1   1.035671
12     272.101196  -70.086845      17        1   0.802800
0      -53.974472   60.768681      18        1   0.586825
11    -216.418228 -196.161316      19        1   0.353556
2      103.390175 -108.375671      20        1   0.353556, topic_info=     Term       Freq      Total Category  logprob  loglift
621    良い  23.000000  23.000000  Default  30.0000  30.0000
439    授業  38.000000  38.000000  Default  29.0000  29.0000
662    試験  28.000000  28.000000  Default  28.0000  28.0000
546    特に   8.000000   8.000000  Default  27.0000  27.0000
80    とても  24.000000  24.000000  Default  26.0000  26.0000
..    ...        ...        ...      ...      ...      ...
23     いう   0.009901  19.709635  Topic20  -6.6065  -1.9513
218  休み時間   0.009901   2.204622  Topic20  -6.6065   0.2392
93     まだ   0.009901   3.498658  Topic20  -6.6065  -0.2226
648    解説   0.009901   3.726482  Topic20  -6.6065  -0.2857
52    しょう   0.009901   1.094344  Topic20  -6.6065   0.9396

[946 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
0         2  0.215741      cm
0         3  0.431483      cm
0         6  0.215741      cm
4         2  0.552671  github
8         9  0.585832      it
...     ...       ...     ...
733       1  0.704070      願う
733       3  0.176017      願う
734       1  0.918969      高い
736       1  0.923311      高校
737       3  0.892934     魅力的

[513 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[15, 17, 9, 11, 16, 10, 18, 5, 4, 7, 19, 20, 8, 2, 6, 14, 13, 1, 12, 3])